In [3]:
song_name = 'burg_spinnerlied.mid'

In [ ]:
import numpy as np
import midi
import tensorflow as tf
import glob
import IPython
import sys
from keras.callbacks import ModelCheckpoint
import keras
import os
from keras.models import load_model

In [5]:
lowerBound = 24
upperBound = 102
span = upperBound-lowerBound

In [6]:
def midiToNoteStateMatrix(midifile, squash=True, span=span):
    pattern = midi.read_midifile(midifile)
    timeleft = [track[0].tick for track in pattern]
    posns = [0 for track in pattern]

    statematrix = []
    time = 0

    state = [[0,0] for x in range(span)]
    statematrix.append(state)
    condition = True
    while condition:
        if time % (pattern.resolution / 4) == (pattern.resolution / 8):
            # Crossed a note boundary. Create a new state, defaulting to holding notes
            oldstate = state
            state = [[oldstate[x][0],0] for x in range(span)]
            statematrix.append(state)
        
        for i in range(len(timeleft)): #For each track
            if not condition:
                break
            while timeleft[i] == 0:
                track = pattern[i]
                pos = posns[i]

                evt = track[pos]
                if isinstance(evt, midi.NoteEvent):
                    if (evt.pitch < lowerBound) or (evt.pitch >= upperBound):
                        pass
#                         print("Note {} at time {} out of bounds (ignoring)".format(evt.pitch, time))
                    else:
                        if isinstance(evt, midi.NoteOffEvent) or evt.velocity == 0:
                            state[evt.pitch-lowerBound] = [0, 0]
                        else:
                            state[evt.pitch-lowerBound] = [1, 1]
                elif isinstance(evt, midi.TimeSignatureEvent):
                    if evt.numerator not in (2, 4):
                        # We don't want to worry about non-4 time signatures. Bail early!
                        # print "Found time signature event {}. Bailing!".format(evt)
                        out =  statematrix
                        condition = False
                        break
                try:
                    timeleft[i] = track[pos + 1].tick
                    posns[i] += 1
                except IndexError:
                    timeleft[i] = None

            if timeleft[i] is not None:
                timeleft[i] -= 1

        if all(t is None for t in timeleft):
            break

        time += 1

    S = np.array(statematrix)
    statematrix = np.hstack((S[:, :, 0], S[:, :, 1]))
    statematrix = np.asarray(statematrix).tolist()
    return statematrix

In [7]:
def noteStateMatrixToMidi(statematrix, name="example", span=span):
    statematrix = np.array(statematrix)
    if not len(statematrix.shape) == 3:
        statematrix = np.dstack((statematrix[:, :span], statematrix[:, span:]))
    statematrix = np.asarray(statematrix)
    pattern = midi.Pattern()
    track = midi.Track()
    pattern.append(track)
    
    span = upperBound-lowerBound
    tickscale = 45
    
    lastcmdtime = 0
    prevstate = [[0,0] for x in range(span)]
    for time, state in enumerate(statematrix + [prevstate[:]]):  
        offNotes = []
        onNotes = []
        for i in range(span):
            n = state[i]
            p = prevstate[i]
            if p[0] == 1:
                if n[0] == 0:
                    offNotes.append(i)
                elif n[1] == 1:
                    offNotes.append(i)
                    onNotes.append(i)
            elif n[0] == 1:
                onNotes.append(i)
        for note in offNotes:
            track.append(midi.NoteOffEvent(tick=(time-lastcmdtime)*tickscale, pitch=note+lowerBound))
            lastcmdtime = time
        for note in onNotes:
            track.append(midi.NoteOnEvent(tick=(time-lastcmdtime)*tickscale, velocity=40, pitch=note+lowerBound))
            lastcmdtime = time
            
        prevstate = state
    
    eot = midi.EndOfTrackEvent(tick=1)
    track.append(eot)

    midi.write_midifile("{}.midi".format(name), pattern)

In [8]:
#notes = []
#for file in glob.glob('C:/Users/singh/Downloads/Classical-Piano-Composer-master/midi_songs/*.mid'):
#    state_matrix = midiToNoteStateMatrix(file)
#    notes.append(state_matrix)
#print(len(notes))

In [9]:
state_matrix = midiToNoteStateMatrix('C:/Users/singh/Downloads/Music Generation/midi_songs/classical-music-midi/burgm/' + song_name)
state_matrix = np.array(state_matrix)
state_matrix.shape

(657, 156)

In [10]:
input_seq = []
output_seq = []
seq_len = 100
for i in range(state_matrix.shape[0] - seq_len):
    input_seq.append(state_matrix[i:(i+seq_len)])
    output_seq.append(state_matrix[(i+seq_len)])
input_seq = np.array(input_seq)
output_seq = np.array(output_seq)

print("Input array shape is " + str(input_seq.shape))
print("Output array shape is " + str(output_seq.shape))

Input array shape is (557, 100, 156)
Output array shape is (557, 156)


## Using 1 LSTM layer

In [47]:
model_1 = tf.keras.Sequential([
    tf.keras.layers.LSTM(256, input_shape = (input_seq.shape[1], input_seq.shape[2]), recurrent_dropout = 0.3,),
    tf.keras.layers.Dense(output_seq.shape[1], activation  = 'sigmoid'),
    ])
model_1.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['acc'])

In [217]:
filepath_1 = "C:/Users/singh/Downloads/Music Generation/Checkpoints/MIDI/weights_1.best.hdf5"

checkpoint = ModelCheckpoint(filepath_1,
                             monitor='loss', 
                             verbose=0, 
                             save_best_only=True, 
                             mode='min')
callbacks_list = [checkpoint]

model_1.fit(input_seq, output_seq, epochs=200,  batch_size=64, callbacks = callbacks_list)

Train on 557 samples
Epoch 1/200
557/557 [==============================] - 21s 37ms/sample - loss: 15.7990 - acc: 0.0610
Epoch 2/200
557/557 [==============================] - 22s 40ms/sample - loss: 15.4992 - acc: 0.0610
Epoch 3/200
557/557 [==============================] - 24s 43ms/sample - loss: 15.2721 - acc: 0.0610
Epoch 4/200
557/557 [==============================] - 24s 43ms/sample - loss: 15.2226 - acc: 0.0610
Epoch 5/200
557/557 [==============================] - 24s 44ms/sample - loss: 15.1550 - acc: 0.0610
Epoch 6/200
557/557 [==============================] - 26s 47ms/sample - loss: 15.1250 - acc: 0.0610
Epoch 7/200
557/557 [==============================] - 25s 45ms/sample - loss: 15.0644 - acc: 0.0610
Epoch 8/200
557/557 [==============================] - 26s 46ms/sample - loss: 15.0802 - acc: 0.0610
Epoch 9/200
557/557 [==============================] - 26s 46ms/sample - loss: 15.0566 - acc: 0.0610
Epoch 10/200
557/557 [==============================] - 25s 45ms/sampl

557/557 [==============================] - 26s 46ms/sample - loss: 5.1845 - acc: 0.4039
Epoch 162/200
557/557 [==============================] - 26s 46ms/sample - loss: 5.1872 - acc: 0.3986
Epoch 163/200
557/557 [==============================] - 25s 46ms/sample - loss: 5.1878 - acc: 0.4093
Epoch 164/200
557/557 [==============================] - 25s 46ms/sample - loss: 5.1837 - acc: 0.4111
Epoch 165/200
557/557 [==============================] - 26s 47ms/sample - loss: 5.1779 - acc: 0.4111
Epoch 166/200
557/557 [==============================] - 25s 46ms/sample - loss: 5.1774 - acc: 0.4201
Epoch 167/200
557/557 [==============================] - 29s 51ms/sample - loss: 5.1697 - acc: 0.4022
Epoch 168/200
557/557 [==============================] - 26s 47ms/sample - loss: 5.1827 - acc: 0.4363
Epoch 169/200
557/557 [==============================] - 26s 47ms/sample - loss: 5.1748 - acc: 0.4183
Epoch 170/200
557/557 [==============================] - 26s 47ms/sample - loss: 5.1791 - acc: 0

In [49]:
#model_1.save('MIDI_model_1.h5')  # creates a HDF5 file 'my_model.h5'
#del model_1

model_1 = tf.keras.models.load_model('MIDI_model_1.h5')

In [50]:
pred_note_length = 100

start_point = np.random.randint(0, len(input_seq) - 1)
print(start_point)

pattern = input_seq[start_point]
output = []

for i in range(pred_note_length):
    input_pred = np.reshape(pattern, (1, len(pattern), input_seq.shape[2]))
    result = model_1.predict(input_pred, verbose = 0)
    result = (result > 0.3)*1
    output.append([result[0][i] for i in range(result.shape[1])])
    pattern = np.concatenate((pattern,result))
    pattern = pattern[1:len(pattern)]
output = np.array(output)

27


In [51]:
output.sum(axis = 1)

array([2, 4, 3, 2, 2, 8, 5, 2, 2, 6, 3, 0, 2, 5, 1, 0, 1, 2, 2, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 3, 2, 1, 0, 1, 2, 3, 2, 4, 2,
       4, 0, 3, 1, 3, 5, 0, 2, 3, 0, 0, 3, 2, 2, 3, 0, 2, 1, 2, 3, 1, 3,
       1, 3, 0, 1, 3, 3, 3, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       3, 2, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1])

In [52]:
noteStateMatrixToMidi(output, name = 'test_midi_output_1')

## Using 2 LSTM layer

In [16]:
model_2 = tf.keras.Sequential([
    tf.keras.layers.LSTM(256, input_shape = (input_seq.shape[1], input_seq.shape[2]), return_sequences=True, recurrent_dropout = 0.3,),
    tf.keras.layers.LSTM(512, recurrent_dropout = 0.2),
    tf.keras.layers.Dense(512),
    tf.keras.layers.Dense(output_seq.shape[1], activation  = 'sigmoid')
])
model_2.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['acc'])

In [17]:
filepath_2="C:/Users/singh/Downloads/Music Generation/Checkpoints/MIDI/weights_2.best.hdf5"

checkpoint = ModelCheckpoint(filepath_2,
                             monitor='loss', 
                             verbose=0, 
                             save_best_only=True, 
                             mode='min')
callbacks_list = [checkpoint]

model_2.fit(input_seq, output_seq, epochs=200,  batch_size=64, callbacks = callbacks_list)

Train on 557 samples
Epoch 1/200
557/557 [==============================] - 21s 37ms/sample - loss: 16.6783 - acc: 0.1221
Epoch 2/200
557/557 [==============================] - 19s 34ms/sample - loss: 15.4765 - acc: 0.1508
Epoch 3/200
557/557 [==============================] - 19s 35ms/sample - loss: 15.3768 - acc: 0.1113
Epoch 4/200
557/557 [==============================] - 20s 36ms/sample - loss: 15.2285 - acc: 0.1508
Epoch 5/200
557/557 [==============================] - 20s 37ms/sample - loss: 15.0661 - acc: 0.1508
Epoch 6/200
557/557 [==============================] - 19s 34ms/sample - loss: 15.1625 - acc: 0.1382
Epoch 7/200
557/557 [==============================] - 13s 23ms/sample - loss: 14.5053 - acc: 0.0359
Epoch 8/200
557/557 [==============================] - 14s 25ms/sample - loss: 14.2112 - acc: 0.0646
Epoch 9/200
557/557 [==============================] - 14s 25ms/sample - loss: 13.7171 - acc: 0.0682
Epoch 10/200
557/557 [==============================] - 14s 25ms/sampl

557/557 [==============================] - 16s 29ms/sample - loss: 6.0471 - acc: 0.3429
Epoch 82/200
557/557 [==============================] - 16s 29ms/sample - loss: 6.0167 - acc: 0.3788
Epoch 83/200
557/557 [==============================] - 16s 29ms/sample - loss: 5.9618 - acc: 0.4147
Epoch 84/200
557/557 [==============================] - 16s 28ms/sample - loss: 5.9697 - acc: 0.3537
Epoch 85/200
557/557 [==============================] - 16s 29ms/sample - loss: 5.8952 - acc: 0.4039
Epoch 86/200
557/557 [==============================] - 16s 29ms/sample - loss: 5.8857 - acc: 0.4165
Epoch 87/200
557/557 [==============================] - 16s 29ms/sample - loss: 5.8774 - acc: 0.3465
Epoch 88/200
557/557 [==============================] - 16s 29ms/sample - loss: 5.8506 - acc: 0.3860
Epoch 89/200
557/557 [==============================] - 16s 29ms/sample - loss: 5.7766 - acc: 0.4022
Epoch 90/200
557/557 [==============================] - 17s 31ms/sample - loss: 5.7185 - acc: 0.3914
Epo

557/557 [==============================] - 17s 31ms/sample - loss: 5.1708 - acc: 0.4237
Epoch 162/200
557/557 [==============================] - 18s 32ms/sample - loss: 5.1654 - acc: 0.4129
Epoch 163/200
557/557 [==============================] - 16s 30ms/sample - loss: 5.1694 - acc: 0.4111
Epoch 164/200
557/557 [==============================] - 17s 31ms/sample - loss: 5.1625 - acc: 0.4309
Epoch 165/200
557/557 [==============================] - 17s 30ms/sample - loss: 5.1658 - acc: 0.4578
Epoch 166/200
557/557 [==============================] - 17s 31ms/sample - loss: 5.1603 - acc: 0.4381
Epoch 167/200
557/557 [==============================] - 16s 29ms/sample - loss: 5.1603 - acc: 0.3896
Epoch 168/200
557/557 [==============================] - 16s 28ms/sample - loss: 5.1647 - acc: 0.4093
Epoch 169/200
557/557 [==============================] - 16s 29ms/sample - loss: 5.1593 - acc: 0.4488
Epoch 170/200
557/557 [==============================] - 16s 29ms/sample - loss: 5.1624 - acc: 0

In [18]:
#model_2.save('MIDI_model_2.h5')  # creates a HDF5 file 'my_model.h5'
#del model_2

model_2 = tf.keras.models.load_model('MIDI_model_2.h5')

In [19]:
pred_note_length = 100

start_point = np.random.randint(0, len(input_seq) - 1)
print(start_point)

pattern = input_seq[start_point]
output = []

for i in range(pred_note_length):
    input_pred = np.reshape(pattern, (1, len(pattern), input_seq.shape[2]))
    result = model_2.predict(input_pred, verbose = 0)
    result = (result > 0.3)*1
    output.append([result[0][i] for i in range(result.shape[1])])
    pattern = np.concatenate((pattern,result))
    pattern = pattern[1:len(pattern)]
output = np.array(output)

131


In [20]:
output.sum(axis = 1)

array([3, 4, 3, 2, 2, 8, 5, 2, 2, 8, 4, 2, 2, 8, 5, 2, 2, 6, 4, 2, 2, 8,
       5, 2, 2, 4, 3, 2, 2, 8, 5, 2, 2, 6, 2, 1, 1, 3, 1, 1, 2, 6, 5, 3,
       3, 6, 5, 3, 3, 4, 4, 3, 3, 6, 5, 3, 3, 4, 4, 3, 3, 8, 6, 3, 3, 4,
       3, 2, 2, 8, 5, 2, 2, 8, 4, 2, 2, 8, 5, 2, 2, 6, 4, 2, 2, 8, 5, 2,
       2, 4, 3, 2, 2, 8, 5, 2, 2, 6, 2, 1])

In [23]:
noteStateMatrixToMidi(output, name = 'test_midi_output_2')

## Using 3 LSTM layer

In [24]:
model_3 = tf.keras.Sequential([
    tf.keras.layers.LSTM(256, input_shape = (input_seq.shape[1], input_seq.shape[2]), return_sequences=True, recurrent_dropout = 0.3,),
    tf.keras.layers.LSTM(512, return_sequences=True, recurrent_dropout = 0.3),
    tf.keras.layers.LSTM(256, recurrent_dropout = 0.2),
    tf.keras.layers.Dense(512),
    tf.keras.layers.Dense(output_seq.shape[1], activation  = 'sigmoid')
])
model_3.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['acc'])

In [25]:
filepath_3 = "C:/Users/singh/Downloads/Music Generation/Checkpoints/MIDI/weights_3-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"

checkpoint = ModelCheckpoint(filepath_3,
                             monitor='loss', 
                             verbose=0, 
                             save_best_only=True, 
                             mode='min')
callbacks_list = [checkpoint]

model_3.fit(input_seq, output_seq, epochs=200,  batch_size=64, callbacks = callbacks_list)

Train on 557 samples
Epoch 1/200
557/557 [==============================] - 25s 45ms/sample - loss: 16.6245 - acc: 0.0987
Epoch 2/200
557/557 [==============================] - 24s 43ms/sample - loss: 15.4200 - acc: 0.0610
Epoch 3/200
557/557 [==============================] - 23s 42ms/sample - loss: 15.2475 - acc: 0.0592
Epoch 4/200
557/557 [==============================] - 24s 43ms/sample - loss: 15.1870 - acc: 0.0610
Epoch 5/200
557/557 [==============================] - 24s 43ms/sample - loss: 15.1846 - acc: 0.0610
Epoch 6/200
557/557 [==============================] - 24s 44ms/sample - loss: 15.1318 - acc: 0.1364
Epoch 7/200
557/557 [==============================] - 24s 44ms/sample - loss: 15.0849 - acc: 0.1508
Epoch 8/200
557/557 [==============================] - 24s 44ms/sample - loss: 15.0739 - acc: 0.1508
Epoch 9/200
557/557 [==============================] - 25s 44ms/sample - loss: 15.0712 - acc: 0.1508
Epoch 10/200
557/557 [==============================] - 24s 44ms/sampl

557/557 [==============================] - 26s 47ms/sample - loss: 8.4483 - acc: 0.2729
Epoch 82/200
557/557 [==============================] - 28s 51ms/sample - loss: 8.2648 - acc: 0.2765
Epoch 83/200
557/557 [==============================] - 28s 50ms/sample - loss: 8.2029 - acc: 0.2729
Epoch 84/200
557/557 [==============================] - 27s 48ms/sample - loss: 8.0608 - acc: 0.2675
Epoch 85/200
557/557 [==============================] - 28s 49ms/sample - loss: 7.9863 - acc: 0.2478
Epoch 86/200
557/557 [==============================] - 29s 52ms/sample - loss: 7.8430 - acc: 0.2639
Epoch 87/200
557/557 [==============================] - 28s 50ms/sample - loss: 7.8210 - acc: 0.2783
Epoch 88/200
557/557 [==============================] - 27s 48ms/sample - loss: 7.6283 - acc: 0.2729
Epoch 89/200
557/557 [==============================] - 28s 50ms/sample - loss: 7.5310 - acc: 0.2962
Epoch 90/200
557/557 [==============================] - 28s 50ms/sample - loss: 7.5537 - acc: 0.2747
Epo

557/557 [==============================] - 26s 46ms/sample - loss: 5.3095 - acc: 0.3680
Epoch 162/200
557/557 [==============================] - 26s 46ms/sample - loss: 5.2957 - acc: 0.4219
Epoch 163/200
557/557 [==============================] - 26s 46ms/sample - loss: 5.2834 - acc: 0.3806
Epoch 164/200
557/557 [==============================] - 26s 46ms/sample - loss: 5.2809 - acc: 0.3429
Epoch 165/200
557/557 [==============================] - 25s 46ms/sample - loss: 5.2837 - acc: 0.3429
Epoch 166/200
557/557 [==============================] - 26s 46ms/sample - loss: 5.2784 - acc: 0.4022
Epoch 167/200
557/557 [==============================] - 26s 46ms/sample - loss: 5.2941 - acc: 0.3429
Epoch 168/200
557/557 [==============================] - 25s 46ms/sample - loss: 5.3164 - acc: 0.3788
Epoch 169/200
557/557 [==============================] - 26s 46ms/sample - loss: 5.3126 - acc: 0.3537
Epoch 170/200
557/557 [==============================] - 25s 46ms/sample - loss: 5.2986 - acc: 0

In [26]:
#model_3.save('MIDI_model_3.h5')  # creates a HDF5 file 'my_model.h5'
#del model_3

model_3 = tf.keras.models.load_model('MIDI_model_3.h5')

In [37]:
pred_note_length = 100

start_point = np.random.randint(0, len(input_seq) - 1)
print(start_point)

pattern = input_seq[start_point]
output = []

for i in range(pred_note_length):
    input_pred = np.reshape(pattern, (1, len(pattern), input_seq.shape[2]))
    result = model_3.predict(input_pred, verbose = 0)
    result = (result > 0.2)*1
    output.append([result[0][i] for i in range(result.shape[1])])
    pattern = np.concatenate((pattern,result))
    pattern = pattern[1:len(pattern)]
output = np.array(output)

216


In [39]:
noteStateMatrixToMidi(output, name = 'test_midi_output_3')